In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1,2,3'

In [2]:
import json
from gpt_2.src import model, encoder

with open('bahasa-vocab.json', 'r') as f:
    en = json.load(f)
with open('bahasa-merges.txt', 'r', encoding="utf-8") as f:
    bpe_data = f.read()
    
bpe_merges = [tuple(merge_str.split()) for merge_str in bpe_data.split('\n')[1:-1]]
enc_malay = encoder.Encoder(
    encoder=en,
    bpe_merges=bpe_merges,
)

/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/husein/.local/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:519: 

In [3]:
import numpy as np

In [4]:
import gpt_2_simple

In [5]:
from glob import glob

files = glob('../bert/dumping-*.txt')
files = [i for i in files if 'twitter' not in i and 'instagram' not in i and 'combined' not in i]
files

['../bert/dumping-parliament.txt',
 '../bert/dumping-iium.txt',
 '../bert/dumping-wiki.txt',
 '../bert/dumping-news.txt',
 '../bert/dumping-watpadd.txt',
 '../bert/dumping-pdf.txt']

In [6]:
splitted = glob('split.*')
files = files + splitted
files

['../bert/dumping-parliament.txt',
 '../bert/dumping-iium.txt',
 '../bert/dumping-wiki.txt',
 '../bert/dumping-news.txt',
 '../bert/dumping-watpadd.txt',
 '../bert/dumping-pdf.txt',
 'split.txtao',
 'split.txtat',
 'split.txtaj',
 'split.txtad',
 'split.txtaz',
 'split.txtap',
 'split.txtbb',
 'split.txtbd',
 'split.txtag',
 'split.txtau',
 'split.txtav',
 'split.txtah',
 'split.txtaq',
 'split.txtab',
 'split.txtai',
 'split.txtba',
 'split.txtax',
 'split.txtar',
 'split.txtac',
 'split.txtam',
 'split.txtas',
 'split.txtae',
 'split.txtaw',
 'split.txtay',
 'split.txtan',
 'split.txtak',
 'split.txtaa',
 'split.txtal',
 'split.txtaf',
 'split.txtbc']

In [7]:
import gpt_2_simple

gpt_2_simple.get_available_gpus()

['/device:GPU:0', '/device:GPU:1', '/device:GPU:2']

In [8]:
from gpt_2_simple.src import model, sample, encoder, memory_saving_gradients
from gpt_2_simple.src.load_dataset import load_dataset, Sampler
from gpt_2_simple.src.accumulate import AccumulatingOptimizer

In [9]:
hparams = model.default_hparams()
with open('models/345M/hparams.json') as f:
    hparams.override_from_dict(json.load(f))
    
hparams.__dict__

{'n_vocab': 50257, 'n_ctx': 1024, 'n_embd': 1024, 'n_head': 16, 'n_layer': 24}

In [10]:
use_memory_saving_gradients = True
only_train_transformer_layers = True
accumulate_gradients = 1

In [11]:
import tqdm

combine = 50000

token_chunks = []
raw_text = ''
paths = files

for path in tqdm.tqdm(paths):
    with open(path, 'r', encoding='utf8', errors='ignore') as fp:
        raw_text += fp.read()
    if len(raw_text) >= combine:
        tokens = np.stack(enc_malay.encode(raw_text))
        token_chunks.append(tokens)
        raw_text = ''
    else:
        raw_text += '<|endoftext|>'

100%|██████████| 36/36 [1:05:25<00:00, 109.05s/it]


In [14]:
len(token_chunks)

36

In [17]:
import pickle

with open('dataset.pkl', 'wb') as fopen:
    pickle.dump(token_chunks, fopen)

In [15]:
token_chunks[0][:100]

array([18111,    16, 18111,  4028,  9904,    16, 10427, 18784,   202,
        1928,   323,  3071,  7485,  1378, 18600,  1378,  4322,  9198,
         202,   202, 35594,   557,   725,   343,  7208,   805,  3679,
        2815, 13254, 17746,   340, 12250,    54,  9229,   747,    64,
         202, 20216,   551,    41, 17134,  6897,  6260,   847,  2113,
         202, 46032,   699, 25654,  1942,   202, 26778,  2405,   202,
         202,  1439, 12296,   557,  1046,   202, 38618,  9904,   281,
        6683,  4638,   202, 10427, 10003,   202, 13120,   345, 24478,
         499,  3467, 14036, 22143, 17294,  5229,   202,    37,   452,
        5819, 37776,   340,   295,   747,   202, 23658,  7317, 42263,
        3571,  4909, 17241,   311, 16059,   202,  8393,  1344,  8444,
        3699])